In [1]:
import requests
import time
import datetime
import json
import pandas as pd

#pd.set_option('display.max_rows', 500)

In [2]:
##Getting the time of yesterday
ts = time.time()
yday= datetime.datetime.fromtimestamp(ts) 
yday = yday.replace(hour=23,minute=59,second=59,microsecond=59)

start = int((yday - datetime.timedelta(days=2)).timestamp())
yday = int(yday.timestamp())

In [3]:
##Scraping from CoinMarketCap the candle value of bitcoin for yesterday
api_url= f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/historical?limit=500&id=1&convertId=2781&timeStart={start}&timeEnd={yday}'
r = requests.get(api_url)
data = []
for item in r.json()['data']['quotes']:
    openp = item['quote']['open']
    close = item['quote']['close']
    volume =item['quote']['volume']
    date=item['quote']['timestamp']
    high=item['quote']['high']
    low=item['quote']['low']
    data.append([date,openp,high,low,close,volume])

cols = ["date","open","high","low","close","volume"]

df = pd.DataFrame(data, columns= cols)
df

,date,open,high,low,close,volume
0,2022-06-06T23:59:59.999Z,29910.283724,31693.290111,29894.18733,31370.671948,3.194734e+10


In [4]:
##Add the new data to the database of bitcoin candlesticks
#Due to cap limit of request (100 rows) scrapped the data from last year to now
#Airflow appliance assure that the new value will be added each day
df2 = pd.read_csv("C:\\Users\\hp\\Desktop\\Cv_project\\NLP_Predict\\raw_btc\\raw_btc.csv")
df3 = pd.concat([df2,df],axis=0, ignore_index=True)
df3.sort_values('date')

,date,open,high,low,close,volume
155,2021-01-02T23:59:59.999Z,29376.455834,33155.118819,29091.181191,32127.267939,6.786542e+10
156,2021-01-03T23:59:59.999Z,32129.407356,34608.556738,32052.317076,32782.024466,7.866524e+10
157,2021-01-04T23:59:59.999Z,32810.947644,33440.218569,28722.755271,31971.913522,8.116348e+10
158,2021-01-05T23:59:59.999Z,31977.041132,34437.590592,30221.186739,33992.429344,6.754732e+10
159,2021-01-06T23:59:59.999Z,34013.614533,36879.698569,33514.033742,36824.364410,7.528943e+10
...,...,...,...,...,...,...
153,2022-06-03T23:59:59.999Z,30467.806840,30633.034476,29375.689302,29704.391357,2.617555e+10
154,2022-06-04T23:59:59.999Z,29706.139573,29930.563947,29500.006350,29832.914225,1.658837e+10
519,2022-06-05T23:59:59.999Z,29835.118132,30117.743447,29574.449531,29906.661748,1.726409e+10
520,2022-06-06T23:59:59.999Z,29910.283724,31693.290111,29894.187330,31370.671948,3.194734e+10


In [5]:
#Write it down for storage
df3.to_csv("C:\\Users\\hp\\Desktop\\Cv_project\\NLP_Predict\\raw_btc\\raw_btc.csv",index=False)

In [6]:
##Function used to ease scrappinp
#Provide timestamps from start date to end date
#With a 100 days separated
#def timeframe(start, end):
    #timeframe = []
    #day_seconds = 60*60*24*100
    #for i in range(start,end+1,day_seconds):
        #timeframe.append(int(datetime.datetime.fromtimestamp(i).timestamp()))
    #return timeframe

#startt = 1609541999
#endd = 1640991599

#test = timeframe(startt,endd) + [endd]
#test